# RNN 모형의 비교
IMDB 데이터

- simple RNN
- LSTM
- GRU
- Bidirectional RNN

In [1]:
import numpy as np
from tensorflow.keras.datasets import imdb
max_features=10000 # 9999개의 단어 사용
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=max_features)

print(type(x_train));print(type(x_test))
print(type(y_train));print(type(y_test))
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(25000,)
(25000,)
(25000,)


In [3]:
# y_train과 y_test 분포
unique,count=np.unique(y_train,return_counts=True)
print(dict(zip(unique,count)))
unique,count=np.unique(y_test,return_counts=True)
print(dict(zip(unique,count)))

{0: 12500, 1: 12500}
{0: 12500, 1: 12500}


In [4]:
print(x_train[0])
print(len(x_train[0]))
wordindex=imdb.get_word_index()
print(len(wordindex)) # 단어 수
print(type(wordindex)) # 자료형

for key,value in wordindex.items():
    if value==1500:
        print(key)

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
218
88584
<class 'dict'>
lacks


1500로 인덱스된 단어는 lacks 이며 lacks는 IMDB에 있는 단어 중 1500번째로 빈도가 높다.

In [5]:
# 인덱스 계열을 원래 문장으로 환원
exchindex=dict([(value,key) for (key,value) in wordindex.items()]) # 자리 바꿈(swap)
review=' '.join([exchindex.get(i-3,'!') for i in x_train[0]]) # 계열의 시작, padding, unkown 제외
print(review)

! this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ! is an amazing actor and now the same being director ! father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ! and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ! to the two little boy's that played the ! of norman and paul they were just brilliant children are often left out of the ! list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

### Padding

In [8]:
from tensorflow.keras.preprocessing import sequence
lenmax=700 # 700개의 계열로 padding
input_train=sequence.pad_sequences(x_train,maxlen=lenmax) # 패딩된 input
input_test=sequence.pad_sequences(x_test,maxlen=lenmax)
print(input_train.shape)
print(input_test.shape)

(25000, 700)
(25000, 700)


## 1. Simple RNN

In [9]:
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN
from tensorflow.keras.models import Sequential

srnn=Sequential()
srnn.add(Embedding(max_features,32,input_shape=(700,)))
srnn.add(SimpleRNN(32))
srnn.add(Dense(1,activation='sigmoid'))
srnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 700, 32)           320000    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 322,113
Trainable params: 322,113
Non-trainable params: 0
_________________________________________________________________


In [10]:
srnn.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
srnn.fit(input_train,y_train,epochs=10,batch_size=128)

train_loss_acc=srnn.evaluate(input_train,y_train)
test_loss_acc=srnn.evaluate(input_test,y_test)

print(train_loss_acc)
print(test_loss_acc)

Epoch 1/10
196/196 [==============================] - 58s 280ms/step - loss: 0.5839 - acc: 0.6819
Epoch 2/10
196/196 [==============================] - 57s 291ms/step - loss: 0.3590 - acc: 0.8522
Epoch 3/10
196/196 [==============================] - 56s 288ms/step - loss: 0.2938 - acc: 0.8824
Epoch 4/10
196/196 [==============================] - 55s 281ms/step - loss: 0.2324 - acc: 0.9114
Epoch 5/10
196/196 [==============================] - 54s 278ms/step - loss: 0.1993 - acc: 0.9260
Epoch 6/10
196/196 [==============================] - 53s 272ms/step - loss: 0.1484 - acc: 0.9461
Epoch 7/10
196/196 [==============================] - 55s 278ms/step - loss: 0.1105 - acc: 0.9616
Epoch 8/10
196/196 [==============================] - 54s 274ms/step - loss: 0.0814 - acc: 0.9719
Epoch 9/10
196/196 [==============================] - 54s 273ms/step - loss: 0.0585 - acc: 0.9818
Epoch 10/10
782/782 [==============================] - 35s 45ms/step - loss: 0.5611 - acc: 0.8342
[0.01345629990100860

과대적합 

## 2. LSTM

In [7]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
m_lstm=Sequential()
m_lstm.add(Embedding(max_features,32, input_length=700))
m_lstm.add(LSTM(32))
m_lstm.add(Dense(1,activation='sigmoid'))
m_lstm.summary()

m_lstm.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
m_lstm.fit(input_train,y_train,epochs=10,batch_size=128)

train_loss_acc=m_lstm.evaluate(input_train,y_train)
test_loss_acc=m_lstm.evaluate(input_test,y_test)
print(train_loss_acc)
print(test_loss_acc)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 700, 32)           320000    
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 328,353
Trainable params: 328,353
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
196/196 [==============================] - 55s 275ms/step - loss: 0.5759 - acc: 0.6932
Epoch 2/10
196/196 [==============================] - 56s 285ms/step - loss: 0.2859 - acc: 0.8899
Epoch 3/10
196/196 [==============================] - 58s 294ms/step - loss: 0.2254 - acc: 0.9164
Epoch 4/10
196/196 [==============================] - 73s 374ms/step - loss: 0.2042 

## 3. GRU

In [8]:
from tensorflow.keras.layers import GRU
from tensorflow.keras.models import Sequential
m_gru=Sequential()
m_gru.add(Embedding(max_features,32))
m_gru.add(GRU(32))
m_gru.add(Dense(1,activation='sigmoid'))
m_gru.summary()

m_gru.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
m_gru.fit(input_train,y_train,epochs=10,batch_size=128)

train_loss_acc=m_gru.evaluate(input_train,y_train)
test_loss_acc=m_gru.evaluate(input_test,y_test)
print(train_loss_acc)
print(test_loss_acc)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
gru (GRU)                    (None, 32)                6336      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 326,369
Trainable params: 326,369
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
196/196 [==============================] - 47s 232ms/step - loss: 0.6106 - acc: 0.6514
Epoch 2/10
196/196 [==============================] - 48s 244ms/step - loss: 0.3120 - acc: 0.8737
Epoch 3/10
196/196 [==============================] - 53s 272ms/step - loss: 0.2467 - acc: 0.9052
Epoch 4/10
196/196 [==============================] - 66s 337ms/step - loss: 0.2199 

## 4. RNN 모형 반복
- 2개의LSTM 모형 연결

In [7]:
from tensorflow.keras.layers import GRU, Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
model=Sequential()
model.add(Embedding(max_features,32, input_length=700))
model.add(LSTM(32, dropout=0.5, recurrent_dropout=0.5, return_sequences=True))
model.add(LSTM(64, dropout=0.5, recurrent_dropout=0.5))
model.add(Dense(1,activation='sigmoid'))
model.summary()

model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
model.fit(input_train,y_train,epochs=10,batch_size=128)
train_loss_acc=model.evaluate(input_train,y_train)
test_loss_acc=model.evaluate(input_test,y_test)
print(train_loss_acc)
print(test_loss_acc)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 700, 32)           320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 700, 32)           8320      
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 353,217
Trainable params: 353,217
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
196/196 [==============================] - 1216s 6s/step - loss: 0.5861 - acc: 0.6640
Epoch 2/10
196/196 [==============================] - 284s 1s/step - loss: 0.3291 - acc: 0.8658
Epoch 3/10
196/196 [==============================] -